## Extracción de palabras clave en noticias:

Actualmente la Dirección de Comunicación interna presenta un reporte de las noticias más relevantes en los medios electrónicos (periódicos, revistas, sitios web oficiales de instituciones públicas, entre otros), sin embargo, está utilizando mucho tiempo en la lectura de las noticias y busca eficientizar su proceso. Como primer paso, busca conocer (sin necesidad de leer cada una de las notas periodísticas) cuales de estas comentan al respecto de una misma situación (cuál es el tema de la noticia).

En el archivo de Excel (Ejercicio_notas_periodísticas.xlsx), en la hoja BD, se presenta una pequeña base de datos sobre noticias de diferentes medios electrónicos, ¿cómo darías solución al problema de la Dirección de Comunicación?.

Por ejemplo: En las primeras once noticias se comenta al respecto de "La incapacidad de Pemex de producir Diésel de Ultra Bajo Azufre". En este caso, la Dirección de Comunicación debería recibir estas mismas once noticias indicando en un nuevo campo el tema en común.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Ejercicio_notas_periodísticas.xlsx to Ejercicio_notas_periodísticas.xlsx


##Bibliotecas


In [ ]:
!pip install scispacy
!spacy download es_core_news_md

     |████████████████████████████████| 44 kB 2.3 MB/s 
     |████████████████████████████████| 71 kB 6.5 MB/s 
     |████████████████████████████████| 13.5 MB 38.6 MB/s 
     |████████████████████████████████| 6.3 MB 49.0 MB/s 
     |████████████████████████████████| 188 kB 48.0 MB/s 
     |████████████████████████████████| 451 kB 46.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 628 kB 46.8 MB/s 
     |████████████████████████████████| 10.1 MB 73.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall

In [ ]:
import re
import nltk
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import gensim
from gensim import models, corpora
from gensim.models import CoherenceModel

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Cargando datos

In [ ]:
df_data = pd.read_excel("Ejercicio_notas_periodísticas.xlsx")
df_data.head(5)

,id,titulo,texto,dominio
0,1,La Jornada - Sin diésel DUBA no puede renovars...,Ciudad de México. Ante la incapacidad de Petró...,jornada.com.mx
1,2,Ajustar NOM–044 demandan a la Semarnat titular...,Presidentes y directores ejecutivos de 12 orga...,automotores-rev.com
2,3,Disminuye disposición de diesel más limpio,Las principales asociaciones de transporte de ...,eluniversal.com.mx
3,4,"A la baja, oferta de diésel de ultra bajo azuf...",Ante la incapacidad de Petróleos Mexicanos (P...,eleconomista.com.mx
4,5,Disminuye disponibilidad de diésel UBA en los ...,Las principales asociaciones de transporte de ...,eluniversal.com.mx


##Preprocesamiento de los textos del dataset

*  Conversión de todas las palabras a minúsculas
*  Eliminación de stopwords
*  Tokenización
*  Obtención de Bigramas

In [ ]:
#convertir todas las palabras de los textos a minusculas
df_data['texto'] = df_data['texto'].str.lower()

#eliminar stopwords
spacy.prefer_gpu()
nlp = spacy.load('es_core_news_md')

stop_words_sp = list(spacy.lang.es.stop_words.STOP_WORDS)
stop_words_sp.extend(['a','versus','vía', 'acá', 'allá', 'atrás','cuan', 'etc', 'sr', 'sra','sres', 'y'])  

def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in stop_words_sp:
      result.append(token)   
  r = " ".join(result)
  return r

df_data['texto_sin_stopwords'] = df_data['texto'].apply(preprocess)

#tokenizacion del texto
df_data['textos_tokenizados'] = df_data.apply(lambda row: nltk.word_tokenize(row['texto_sin_stopwords']), axis=1)
documents_tokenized = list(df_data['textos_tokenizados'])

#obtencion de bigramas
bigram = gensim.models.Phrases(documents_tokenized, min_count=5, threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
documents_bigram_tokenized = [bigram_mod[doc] for doc in documents_tokenized]

df_data['textos_tokenizados'] = documents_bigram_tokenized
documents_tokenized = list(df_data['textos_tokenizados'])

In [ ]:
df_data.head(5)

,id,titulo,texto,dominio,texto_sin_stopwords,textos_tokenizados
0,1,La Jornada - Sin diésel DUBA no puede renovars...,ciudad de méxico. ante la incapacidad de petró...,jornada.com.mx,ciudad méxico incapacidad petróleos mexicanos ...,"[ciudad, méxico, incapacidad, petróleos_mexica..."
1,2,Ajustar NOM–044 demandan a la Semarnat titular...,presidentes y directores ejecutivos de 12 orga...,automotores-rev.com,presidentes directores ejecutivos organismos c...,"[presidentes, directores, ejecutivos, organism..."
2,3,Disminuye disposición de diesel más limpio,las principales asociaciones de transporte de ...,eluniversal.com.mx,principales asociaciones transporte carga pasa...,"[principales, asociaciones, transporte, carga,..."
3,4,"A la baja, oferta de diésel de ultra bajo azuf...",ante la incapacidad de petróleos mexicanos (p...,eleconomista.com.mx,incapacidad petróleos mexicanos pemex producir...,"[incapacidad, petróleos_mexicanos, pemex, prod..."
4,5,Disminuye disponibilidad de diésel UBA en los ...,las principales asociaciones de transporte de ...,eluniversal.com.mx,principales asociaciones transporte carga pasa...,"[principales, asociaciones, transporte, carga,..."


## Obtención de representación Bolsa de palabras

In [ ]:
#obtener vocabulario del corpus y mapearlo a un id
id2word = corpora.Dictionary(documents_tokenized)

#obtener representacion bag of words de cada documento
corpus = [id2word.doc2bow(d) for d in documents_tokenized]

##Experimentos con multiples configuraciones

In [ ]:
# Num_topics
topics = range(2, 4, 1)

# Alpha 
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Eta
eta = list(np.arange(0.01, 1, 0.3))
eta.append('symmetric')

print("Posibles valores para el parámetro 'num_topics':", list(topics))
print("Posibles valores para el parámetro 'alpha':", alpha)
print("Posibles valores para el parámetro 'eta':", eta)

Posibles valores para el parámetro 'num_topics': [2, 3]
Posibles valores para el parámetro 'alpha': [0.01, 0.31, 0.61, 0.9099999999999999, 'symmetric', 'asymmetric']
Posibles valores para el parámetro 'eta': [0.01, 0.31, 0.61, 0.9099999999999999, 'symmetric']


In [ ]:
model_result = {'No.Topicos':[], 'Alpha': [], 'Eta': [], 'Coherence':[]}

best_cv = 0
best_no_topics = 0
best_alpha = 0
best_eta = 0

for i in topics:
  for j in alpha:
    for k in eta:
      lda_model = models.LdaModel(corpus, id2word=id2word, num_topics=i, passes=10, alpha=j, eta=k)
      coherence_model = CoherenceModel(model=lda_model, texts = documents_tokenized, dictionary=id2word, coherence='c_v')
      cv = coherence_model.get_coherence()
      model_result['No.Topicos'].append(i)
      model_result['Alpha'].append(j)
      model_result['Eta'].append(k)
      model_result['Coherence'].append(cv)

      if cv > best_cv:
        best_cv = cv
        best_no_topics = i
        best_alpha = j
        best_eta = k

df_models_evaluations = pd.DataFrame(model_result) 

In [ ]:
print("Mejor Configuracion: No.Topicos={}, alpha={}, eta={}".format(best_no_topics, best_alpha, best_eta))
df_models_evaluations.head(5)

Mejor Configuracion: No.Topicos=2, alpha=0.31, eta=0.61


,No.Topicos,Alpha,Eta,Coherence
0,2,0.01,0.01,0.678163
1,2,0.01,0.31,0.686144
2,2,0.01,0.61,0.514393
3,2,0.01,0.91,0.653167
4,2,0.01,symmetric,0.503969


In [ ]:
best_lda_model = models.LdaModel(corpus, id2word=id2word, num_topics=best_no_topics, passes=10, alpha=best_alpha, eta=best_eta)

##Obtención de Tópicos

In [ ]:
for i,topic in best_lda_model.show_topics(formatted=True, num_topics=best_no_topics, num_words=5):
  print(str(i)+": "+ topic)
  print()

0: 0.015*"pemex" + 0.011*"incendio" + 0.011*"gas" + 0.010*"méxico" + 0.008*"fuga"

1: 0.014*"duba" + 0.010*"diésel" + 0.008*"pemex" + 0.008*"país" + 0.007*"industria"



In [ ]:
#obtener el topico de cada documento
r=[]
for i in range(df_data.shape[0]):
  temp  = best_lda_model[corpus[i]]
  temp = sorted(temp, key=lambda x: x[1], reverse=True)
  r.append(temp[0][0])
  print("Articulo {} pertenece al topico {} con una probabilidad {}".format(i, temp[0][0], temp[0][1]))

Articulo 0 pertenece al topico 1 con una probabilidad 0.9995818734169006
Articulo 1 pertenece al topico 1 con una probabilidad 0.9992234110832214
Articulo 2 pertenece al topico 1 con una probabilidad 0.9963169097900391
Articulo 3 pertenece al topico 1 con una probabilidad 0.9987390041351318
Articulo 4 pertenece al topico 1 con una probabilidad 0.9984705448150635
Articulo 5 pertenece al topico 1 con una probabilidad 0.9980335831642151
Articulo 6 pertenece al topico 1 con una probabilidad 0.999094545841217
Articulo 7 pertenece al topico 1 con una probabilidad 0.9990639686584473
Articulo 8 pertenece al topico 1 con una probabilidad 0.9991470575332642
Articulo 9 pertenece al topico 1 con una probabilidad 0.9984814524650574
Articulo 10 pertenece al topico 1 con una probabilidad 0.9990777969360352
Articulo 11 pertenece al topico 1 con una probabilidad 0.9989154934883118
Articulo 12 pertenece al topico 1 con una probabilidad 0.9987818598747253
Articulo 13 pertenece al topico 0 con una probabi

##Predicción del tópico de un nuevo artículo

In [ ]:
new_article = "El incendio de Pemex fue provocado por una fuga de gas."
new_article = new_article.lower()

new_article_clean = preprocess(new_article)
new_article_clean_tokenized = nltk.word_tokenize(new_article_clean)

In [ ]:
id2word_new_article = corpora.Dictionary([new_article_clean_tokenized])
new_article_corpus = id2word.doc2bow(new_article_clean_tokenized)
temp1 = best_lda_model[new_article_corpus]
temp1 =  sorted(temp1, key=lambda x: x[1], reverse=True)
print("La oracion nueva pertenece al topico {} con una probabilidad {}".format(temp1[0][0], temp1[0][1]))

La oracion nueva pertenece al topico 0 con una probabilidad 0.9435540437698364


##Resultado

In [ ]:
df_result = pd.read_excel("Ejercicio_notas_periodísticas.xlsx")
df_result['Tópico_Id'] = r

topics = best_lda_model.show_topics(formatted=True, num_topics=best_no_topics, num_words=5)

df_result['Tópico_Descripción'] = ""
for tp, words in topics:
  df_result.loc[df_result.Tópico_Id == tp, "Tópico_Descripción"] = re.sub('[^A-Za-záéíóúñÑ]+', ' ', words)


In [ ]:
df_result

,id,titulo,texto,dominio,Tópico_Id,Tópico_Descripción
0,1,La Jornada - Sin diésel DUBA no puede renovars...,Ciudad de México. Ante la incapacidad de Petró...,jornada.com.mx,1,duba diésel pemex país industria
1,2,Ajustar NOM–044 demandan a la Semarnat titular...,Presidentes y directores ejecutivos de 12 orga...,automotores-rev.com,1,duba diésel pemex país industria
2,3,Disminuye disposición de diesel más limpio,Las principales asociaciones de transporte de ...,eluniversal.com.mx,1,duba diésel pemex país industria
3,4,"A la baja, oferta de diésel de ultra bajo azuf...",Ante la incapacidad de Petróleos Mexicanos (P...,eleconomista.com.mx,1,duba diésel pemex país industria
4,5,Disminuye disponibilidad de diésel UBA en los ...,Las principales asociaciones de transporte de ...,eluniversal.com.mx,1,duba diésel pemex país industria
5,6,Baja disponibilidad de diésel de ultra bajo az...,"De 2018 a 2021, que es el tiempo que lleva And...",elfinanciero.com.mx,1,duba diésel pemex país industria
6,7,La Jornada Maya . Nacional,Con el argumento que el diésel de combustión ...,lajornadamaya.mx,1,duba diésel pemex país industria
7,8,Sector transporte pide retrasar uso de diésel ...,14 de junio 2021 . 11:48 am Argumentando que e...,elceo.com,1,duba diésel pemex país industria
8,9,Alertan transportistas poca disponibilidad de ...,La disponibilidad de Diésel de Ultra bajo Azuf...,elfinanciero.com.mx,1,duba diésel pemex país industria
9,10,Decrece suministro de DUBA para cadena de auto...,Ante la negativa de Petróleos Mexicanos (Peme...,eleconomista.com.mx,1,duba diésel pemex país industria
